In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import time

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/',one_hot=True)
data.test.cls = np.argmax(data.test.labels ,axis =1)
data.validation.cls = np.argmax(data.validation.labels, axis=1)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
img_size = 28
img_size_flat = img_size*img_size
img_shape= (img_size,img_size)
num_channels = 1
num_classes = 10

# MODEL

## TF GRAPH INPUTS

In [5]:
x = tf.placeholder(tf.float32,shape=[None,img_size_flat],name='x')
x_image = tf.reshape(x,[-1,img_size,img_size,num_channels]) 

y_true = tf.placeholder(tf.float32,shape=[None,num_classes],name='y_true')
y_true_cls = tf.arg_max(y_true,dimension=1)

## TF LAYERS

In [6]:
net = x_image
net = tf.layers.conv2d(inputs=net,name='layer_conv1',padding='same',
                       filters=16,kernel_size=5,activation=tf.nn.relu)
layer_conv1 = net
net = tf.layers.max_pooling2d(inputs=net ,pool_size=2,strides=2)
net = tf.layers.conv2d(inputs=net,name='layer_conv2',padding='same',
                       filters=36,kernel_size=5,activation=tf.nn.relu)
layer_conv2= net
net = tf.layers.max_pooling2d(inputs=net,pool_size=2,strides=2)
net = tf.contrib.layers.flatten(net)
net = tf.layers.dense(inputs=net,name='layer_fc1',units=128,activation=tf.nn.relu)
net = tf.layers.dense(inputs=net,name='layer_fc_out',units=num_classes,activation=tf.nn.relu)

In [7]:
print layer_conv1
print layer_conv2
print net

Tensor("layer_conv1/Relu:0", shape=(?, 28, 28, 16), dtype=float32)
Tensor("layer_conv2/Relu:0", shape=(?, 14, 14, 36), dtype=float32)
Tensor("layer_fc_out/Relu:0", shape=(?, 10), dtype=float32)


## Logits
Output of the final layer before softmax is called Logits.

In [8]:
logits = net
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.arg_max(y_pred,dimension=1)

In [9]:
correct_prediction = tf.equal(y_pred_cls,y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

## Loss & Optimizer 

In [10]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_true)
loss = tf.reduce_mean(cross_entropy)

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

In [12]:
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES) :
    print var

Tensor("layer_conv1/kernel/read:0", shape=(5, 5, 1, 16), dtype=float32)
Tensor("layer_conv1/bias/read:0", shape=(16,), dtype=float32)
Tensor("layer_conv2/kernel/read:0", shape=(5, 5, 16, 36), dtype=float32)
Tensor("layer_conv2/bias/read:0", shape=(36,), dtype=float32)
Tensor("layer_fc1/kernel/read:0", shape=(1764, 128), dtype=float32)
Tensor("layer_fc1/bias/read:0", shape=(128,), dtype=float32)
Tensor("layer_fc_out/kernel/read:0", shape=(128, 10), dtype=float32)
Tensor("layer_fc_out/bias/read:0", shape=(10,), dtype=float32)
Tensor("beta1_power/read:0", shape=(), dtype=float32)
Tensor("beta2_power/read:0", shape=(), dtype=float32)
Tensor("layer_conv1/kernel/Adam/read:0", shape=(5, 5, 1, 16), dtype=float32)
Tensor("layer_conv1/kernel/Adam_1/read:0", shape=(5, 5, 1, 16), dtype=float32)
Tensor("layer_conv1/bias/Adam/read:0", shape=(16,), dtype=float32)
Tensor("layer_conv1/bias/Adam_1/read:0", shape=(16,), dtype=float32)
Tensor("layer_conv2/kernel/Adam/read:0", shape=(5, 5, 16, 36), dtype=f

In [13]:
batch_size = 256
def predict_cls(images,labels,cls_true):
    num_images = len(images)
    cls_pred = np.zeros(shape=num_images,dtype=np.int)
    i=0
    while i < num_images :
        j = min(i+batch_size,num_images)
        feed_dict={x:images[i:j,:], y_true:labels[i:j,:]}
        cls_pred[i:j] = session.run(y_pred_cls,feed_dict=feed_dict)
        i=j
    correct = (cls_true==cls_pred)
    return correct,cls_pred

In [14]:
def predict_cls_validation():
    return predict_cls(images = data.validation.images,
                       labels = data.validation.labels,
                       cls_true = data.validation.cls)

In [15]:
def predict_cls_test():
    return predict_cls(images = data.test.images,
                       labels = data.test.labels,
                       cls_true = data.test.cls)

In [16]:
def cls_accuracy(correct):
    correct_sum = correct.sum()
    acc = float(correct_sum) / len(correct)
    return acc, correct_sum

In [17]:
def validation_accuracy():
    correct, _ = predict_cls_validation()
    return cls_accuracy(correct)

In [18]:
def get_weight_variables(layername):
    with tf.variable_scope(layername,reuse=True) :
        variable = tf.get_variable('kernel')
    return variable

## TF SESSION RUN

In [19]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [20]:
saver = tf.train.Saver()
save_path = 'checkpoints/best_validation'

In [21]:
train_batch_size = 128

In [22]:
total_iterations = 0
best_validation_accuracy =0
last_improvement = 0
require_improvement = 300
# for early stopping.

In [23]:
def optimize(num_iterations):
    global total_iterations
    global best_validation_accuracy
    global last_improvement
    
    start_time = time.time()
    for i in range(num_iterations) :
        total_iterations += 1
        x_batch,y_true_batch = data.train.next_batch(train_batch_size)
        feed_dict = {x:x_batch,y_true:y_true_batch}
        session.run(optimizer,feed_dict=feed_dict)
        if (total_iterations % 100 == 0) or (i == num_iterations-1) :
            acc_train = session.run(accuracy,feed_dict=feed_dict)
            acc_validation,_ = validation_accuracy()
            if acc_validation > best_validation_accuracy :
                best_validation_accuracy = acc_validation
                last_improvement = total_iterations
                saver.save(sess=session,save_path=save_path)
                impr_str = '*'
            else :
                impr_str = ''
            print ('Iter {0:>6},Train Accuracy {1:>6.1%},Validation Accuracy {2:>6.1%} {3}').format(total_iterations,acc_train,acc_validation,impr_str)
        
        if total_iterations - last_improvement > require_improvement :
            print ('Early Stopping.')
            break
        
    end_time = time.time()
    print ('Total Usage:',round(end_time-start_time,3),' seconds')
        

In [24]:
test_batch_size = 256
def print_accuracy(show_example_errors= False ,show_confusion_matrix = False) :
    num_test = len(data.test.images)
    cls_pred = np.zeros(shape=num_test,dtype = np.int)
    i=0
    while(i<num_test) :
        j=min(i + test_batch_size, num_test)
        images = data.test.images[i:j,:]
        labels = data.test.labels[i:j,:]
        feed_dict={x:images,y_true:labels}
        cls_pred[i:j] = session.run(y_pred_cls,feed_dict=feed_dict)
        i=j
    cls_true = data.test.cls
    correct = (cls_true==cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum)/num_test   
    print "Accuracy on Test-Set: {0:.1%} ({1} / {2})".format(acc, correct_sum, num_test)

# MODEL RUN

In [25]:
print_accuracy()

Accuracy on Test-Set: 6.0% (597 / 10000)


In [26]:
optimize(1)

Iter      1,Train Accuracy   6.2%,Validation Accuracy   7.4% *
('Total Usage:', 2.826, ' seconds')


In [27]:
print_accuracy()

Accuracy on Test-Set: 8.0% (801 / 10000)


In [28]:
optimize(99)
print_accuracy()

Iter    100,Train Accuracy  70.3%,Validation Accuracy  69.2% *
('Total Usage:', 19.557, ' seconds')
Accuracy on Test-Set: 70.0% (7001 / 10000)


In [29]:
optimize(5000)
print_accuracy()

Iter    200,Train Accuracy  78.1%,Validation Accuracy  81.8% *
Iter    300,Train Accuracy  89.8%,Validation Accuracy  87.3% *
Iter    400,Train Accuracy  91.4%,Validation Accuracy  89.6% *
Iter    500,Train Accuracy  90.6%,Validation Accuracy  91.3% *
Iter    600,Train Accuracy  91.4%,Validation Accuracy  92.6% *
Iter    700,Train Accuracy  95.3%,Validation Accuracy  93.4% *
Iter    800,Train Accuracy  94.5%,Validation Accuracy  93.8% *
Iter    900,Train Accuracy  91.4%,Validation Accuracy  93.9% *
Iter   1000,Train Accuracy  95.3%,Validation Accuracy  95.0% *
Iter   1100,Train Accuracy  96.1%,Validation Accuracy  95.4% *
Iter   1200,Train Accuracy  93.0%,Validation Accuracy  95.5% *
Iter   1300,Train Accuracy  96.1%,Validation Accuracy  95.7% *
Iter   1400,Train Accuracy  94.5%,Validation Accuracy  96.2% *
Iter   1500,Train Accuracy  94.5%,Validation Accuracy  96.5% *
Iter   1600,Train Accuracy  96.1%,Validation Accuracy  96.5% *
Iter   1700,Train Accuracy  97.7%,Validation Accuracy  